# Linear Regression using milliGrad
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mdehling/milligrad/blob/main/demo/linear-regression.ipynb)

Make sure you have the `milliGrad` package installed.

* If you are running this notebook on Google Colab, the first cell will take
  care of it for you.
* If you opened this notebook from within GitHub Codespaces, `milliGrad` should
  already be installed.

In [ ]:
try:
    import milligrad
    MISSING_MILLIGRAD = False
except ImportError:
    MISSING_MILLIGRAD = True

try:
    from google import colab
    RUNNING_ON_COLAB = True
except ImportError:
    RUNNING_ON_COLAB = False

if MISSING_MILLIGRAD and RUNNING_ON_COLAB:
    !pip install -q 'git+https://github.com/mdehling/milligrad.git'
elif MISSING_MILLIGRAD and not RUNNING_ON_COLAB:
    raise ModuleNotFoundError("please install 'milligrad' package")

In [ ]:
from milligrad import Tensor, nn
from milligrad.nn import functional as F
from milligrad.datasets import make_linear

import numpy as np
import matplotlib.pyplot as plt

## Approach 0: An Analytical Solution
Linear regression has a simple analytical solution which minimizes the mean
squared error: $[W\,b]^T = (X^TX)^{-1}X^Ty$.

In [ ]:
p0, p1 = (1,3), (3,2)
x, y = np.hsplit(make_linear(p0, p1, n=100, noise=0.1), 2)

In [ ]:
def solve_linear_regression(x, y):
    n, k = x.shape
    x = np.hstack([x, np.ones((n,1))])
    Wb = y.T @ x @ np.linalg.inv(x.T@x)
    return Wb[:,:-1], Wb[:,-1]

W, b = solve_linear_regression(x, y)

In [ ]:
xs = np.array([[x.min()], [x.max()]])
ys = xs@W + b

plt.scatter(x, y)
plt.plot(xs, ys)
plt.show()

## Approach 1: Using just the Tensor Class
Below we calculate an approximation to the optimal linear regression solution
using gradient descent to repeatedly update the weights to minimize the mean
squared error.  The approach taken here uses the basic `Tensor` class to
calculate the gradients.

In [ ]:
p0, p1 = (1,1), (3,5)
x, y = np.hsplit(make_linear(p0, p1, n=100, noise=0.1), 2)
x, y = Tensor(x, _label='x'), Tensor(y, _label='y')

In [ ]:
# define weights
W = Tensor(np.random.randn(1,1), requires_grad=True, _label='W')
b = Tensor(np.zeros((1,1)), requires_grad=True, _label='b')

lr = 1e-1
for i in range(250):
    # reset gradients to zero
    W._zero_grad()
    b._zero_grad()

    # calculate loss and gradients
    loss = F.mean_squared_error(x@W + b, y)
    loss.backward()

    # update weights
    W.value -= lr * W.grad
    b.value -= lr * b.grad

In [ ]:
xs = np.array([[x.value.min()], [x.value.max()]])
ys = xs@W.value + b.value

plt.scatter(x.value, y.value)
plt.plot(xs, ys)
plt.show()

## Approach 2: Using a Neural Network
In this section we reimplement the previous approach using a `Linear` neural
network layer (perceptron) and the `SGD` optimizer.  The underlying math is
exactly the same.

In [ ]:
p0, p1 = (1,1), (3,-1)
x, y = np.hsplit(make_linear(p0, p1, n=100, noise=0.1), 2)
x, y = Tensor(x, _label='x'), Tensor(y, _label='y')

In [ ]:
model = nn.Linear(1, 1)
opt = nn.optim.SGD(model, lr=1e-1)

for i in range(250):
    opt.zero_grad()
    loss = F.mean_squared_error(model(x), y)
    loss.backward()
    opt.step()

In [ ]:
xs = np.array([[x.value.min()], [x.value.max()]])
ys = model(xs).value

plt.scatter(x.value, y.value)
plt.plot(xs, ys)
plt.show()